In [5]:
import pymongo
import requests
import pandas as pd
#import folium
import random
import json
import time

In [6]:
db = pymongo.MongoClient().simulator

In [7]:
for i in range(100):
    r = requests.post("http://localhost:7500/simulateScenario", json=requests.get("http://localhost:7500/createRandomPlan").json()).json()
    with open('plans.jsonlines', 'a') as f:
        f.write(json.dumps(r))
        f.write('\n')
    time.sleep(10)
    

In [82]:
# read json 
df = pd.read_json( "../../positions.jsonlines", orient="records", lines=True) 
# expand coordinates column
df = df.join(pd.json_normalize(df.coordinates))

In [93]:
# paint positions on map as a line
#random from list of simulationIds

simId = random.choice(df.simulationId.unique())
truckId = random.choice(df[df.simulationId==simId].truckId.unique())

positions = df[(df.simulationId ==simId) & (df.truckId == truckId)]
m = folium.Map(location=[positions.latitude.iat[0] , positions.longitude.iat[0]], zoom_start=10)

folium.PolyLine(positions[["latitude", "longitude"]].values.tolist()).add_to(m)
for index, row in positions.iterrows():
    folium.Marker([row.latitude, row.longitude]).add_to(m)
    
m

In [101]:
events = pd.read_json( "../../simulation.jsonlines", orient="records", lines=True)

In [103]:
events[events.eventType == "Truck arrived"]

,eventDescription,eventTime,eventType,simulationId,truckId
18,"(calle del Marqués de Lozoya, 19, Madrid, [-3....",663000,Truck arrived,19028154-6dbd-47cd-a1f2-22b949695bc5,5030LXK
20,"(calle de Ercilla, 48, Madrid, [-3.70135132692...",694500,Truck arrived,19028154-6dbd-47cd-a1f2-22b949695bc5,3321FBL
24,"(calle de Blanca de Castilla, 11, Madrid, [-3....",740500,Truck arrived,19028154-6dbd-47cd-a1f2-22b949695bc5,5534TPH
29,"(calle de Villamanín, 31, Madrid, [-3.75217778...",817500,Truck arrived,19028154-6dbd-47cd-a1f2-22b949695bc5,1669HSZ
40,"(calle de Ajofrín, 6, Madrid, [-3.625062724069...",973000,Truck arrived,19028154-6dbd-47cd-a1f2-22b949695bc5,3953RLD
...,...,...,...,...,...
5449,"(calle de Toledo, 129, Madrid, [-3.71162311913...",6628500,Truck arrived,81a34fd0-63c1-493d-a799-b846f4abad8b,3321FBL
5454,"(calle de Donoso Cortés, 86, Madrid, [-3.71590...",6765000,Truck arrived,81a34fd0-63c1-493d-a799-b846f4abad8b,6965XLY
5467,"(calle de Fernández de los Ríos, 21, Madrid, [...",10593500,Truck arrived,cd36ad5e-5fe7-429e-93b0-312d440abd73,5534TPH
5469,"(calle de Fernández de los Ríos, 20, Madrid, [...",7133500,Truck arrived,81a34fd0-63c1-493d-a799-b846f4abad8b,6965XLY


In [97]:
# find all plans where simulationId is in the list of simulationIds
plans = list(db["plans"].find({"simulationId": {"$in": df.simulationId.unique().tolist()}}))

In [153]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Extract relevant information and calculate travel delay
trucks = []
for truck in plans[0]['trucks']:
    truck_id = truck['truck_id']
    routes = truck['route']
    truck_events = events[(events.truckId == truck_id) & (events.simulationId == plans[0]["simulationId"])]
    truck_events = truck_events.sort_values(by=['eventTime'])
    event_durations = (truck_events[truck_events.eventType.isin(["Truck arrived", "Truck departed","Truck departed to depot", "Truck ended route"])].eventTime.diff()[1::2]/1000).values
    times = np.array([[x["duration"] for x in routes],event_durations]).T
    delays = times[:,1] - times[:,0]
    times[:,1] =  times[:,1] - times[:,0]
    trucks.append(times)

trucks = np.concatenate(trucks)

# Convert the data into a Pandas DataFrame
df = pd.DataFrame(trucks, columns=['expected_duration','travel_delay'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[[ 'expected_duration']], df['travel_delay'], test_size=0.2, random_state=0)

# Train a Linear Regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regressor.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)


Mean Squared Error: 7.106222867690567
